In [6]:
from requests import get
from math import sqrt
from pymongo import MongoClient
import json
import copy


In [7]:
def get_matches():
    client = MongoClient("mongodb://plp_mongodb:27017")
    db = client['plp']
    matches = list(db["apibase"].find({}))
    return matches
response = get("http://api.localhost/matches")
json_matches = json.loads(response.text)
json_matches[0]

{u'RI_A': -4.064198449734577,
 u'RI_D': -9.953567355091437,
 u'RI_H': -97.88353900019578,
 u'_id': u'2020-2-21/20:30/Bayern Munich - SC Paderborn 07',
 u'date': u'2020-2-21',
 u'hour': u'20:30',
 u'league': u'bundesliga',
 u'match': u'Bayern Munich - SC Paderborn 07',
 u'odd_A': 35.0,
 u'odd_D': 14.0,
 u'odd_H': 1.03,
 u'pred_A': 0.0,
 u'pred_D': 0.0,
 u'pred_H': 0.0,
 u'teamA': u'SC Paderborn 07',
 u'teamH': u'Bayern Munich'}

In [8]:
matches = []

for match in json_matches:
    if (match['RI_A'] > 0):
        match['side'] = "A"
        matches.append(copy.deepcopy(match))
    if (match['RI_D'] > 0):
        match['side'] = "D"
        matches.append(copy.deepcopy(match))
    if (match['RI_H'] > 0):
        match['side'] = "H"
        matches.append(copy.deepcopy(match))
sum = 0
for match in matches:
    sum = sum + match['RI_' + match['side']]
print("avg indicator = ", sum/len(matches))
matches_filtered = sorted(matches, key=lambda d: d["RI_" + d["side"]])[0:10]

('avg indicator = ', 20.37802539355068)


In [9]:
ecart_zero = 0.3
opt = 0.0
risk_opt = 0.0
for risk in range(0,100):

    tot = 0
    for match in matches_filtered:
        e = 0
        side = match['side']
        match['esp'] = match['pred_' + side] * (match['odd_' + side] - 1) + (1 - match['pred_' + side]) * (-1);
        match['var'] = match['pred_' + side] * (match['odd_' + side] - 1 - match['esp']) ** 2 + (1 - match['pred_' + side]) * (-1 - match['esp']) ** 2 + ecart_zero;
        match['cred'] = match['esp']/match['var']**((risk-50)/10.0);
        tot += match['cred']

    esp = 0
    var = 0
    amount = 1
    for match in matches_filtered:
        match['cred'] = amount * match['cred'] / tot
        match['esp'] = match['esp'] * match['cred']
        match['var'] = match['var'] * match['cred'] ** 2
        esp += match['esp']
        var +=  match['var']
    ecart_type = sqrt(var)
    
    if opt < (esp/ecart_type) :
        risk_opt = (risk - 50)/10.0
        opt = esp/ecart_type
print(risk_opt, opt)

(1.0, 0.0887148317377598)


In [10]:
for risk in range(0,6):

    tot = 0
    for match in matches_filtered:
        e = 0
        side = match['side']
        match['esp'] = match['pred_' + side] * (match['odd_' + side] - 1) + (1 - match['pred_' + side]) * (-1);
        match['var'] = match['pred_' + side] * (match['odd_' + side] - 1 - match['esp']) ** 2 + (1 - match['pred_' + side]) * (-1 - match['esp']) ** 2 + ecart_zero;
        match['cred'] = match['esp']/match['var']**(risk_opt-risk);
        tot += match['cred']

    esp = 0
    var = 0
    amount = 1000
    for match in matches_filtered:
        match['cred'] = amount * match['cred'] / tot
        match['esp'] = match['esp'] * match['cred']
        match['var'] = match['var'] * match['cred'] ** 2
        esp += match['esp']
        var +=  match['var']
        #print(match['match'] + " : " + match['side'] + " bet =", match['cred'], " gain= ", match['cred'] * match['odd_' + match['side']])
    ecart_type = sqrt(var)
    print(esp, ecart_type, risk)


(5.274538926054618, 56.12191237276553, 0)
(5.240931540148836, 57.189073845418704, 1)
(5.234027906593369, 61.76002907401376, 2)
(5.259760801071179, 70.51789932244948, 3)
(5.319207028376759, 82.99708120331294, 4)
(5.407380499321146, 97.83249158132972, 5)


In [11]:
data = pd.DataFrame(columns=["date", "id_match", "side", "credit", "odd"])

NameError: name 'pd' is not defined